# Résolution du Problème du Régime de Stigler avec OR-Tools

## Introduction

**Le Problème du Régime de Stigler** :
Dans cette section, nous montrons comment résoudre un problème classique appelé le régime de Stigler, nommé d'après le lauréat du prix Nobel d'économie George Stigler, qui a calculé un moyen économique de répondre aux besoins nutritionnels de base en fonction d'un ensemble d'aliments. Il a posé cela comme un exercice mathématique, et non comme des recommandations alimentaires, bien que l'idée de calculer une nutrition optimale soit devenue à la mode récemment.


## Google OR-Tools 
Google OR-Tools est une suite de logiciels d'optimisation développée par Google. Elle permet de résoudre des problèmes complexes d'optimisation combinatoire tels que la satisfaction de contraintes (CSP), la programmation linéaire (LP), la programmation linéaire mixte (MIP), et bien plus. Dans ce projet, nous nous concentrerons sur la résolution du problème du régime de Stigler en utilisant la technique dîte Solution Linéraire fournie par OR-Tools.


**Solveur de Programmation Linéaire Mixte (MIP)** :
   Ce solveur combine la programmation linéaire avec des variables entières pour résoudre des problèmes d'optimisation. MIP est utilisé pour des problèmes où certaines variables doivent être entières, ce qui est utile dans des scénarios de prise de décision où les solutions doivent être discrètes. La programmation linéaire (LP) est une méthode pour obtenir le meilleur résultat (tel que le profit maximum ou le coût minimum) dans un modèle mathématique dont les exigences sont représentées par des relations linéaires.



Nous allons explorer ce solveur pour résoudre ce problème.

## Importation des Bibliothèques Nécessaires

Nous commençons par importer les bibliothèques nécessaires.





In [1]:
using System;
using System.Collections.Generic;
using System.Collections.ObjectModel;
using System.IO;
using System.Linq;
using System.Text;

#r "nuget: XPlot.Plotly.Interactive"

Installed Packages XPlot.Plotly.Interactive, 4.1.0

Loading extensions from `C:\Users\Administrateur.000\.nuget\packages\xplot.plotly.interactive\4.1.0\lib\net7.0\XPlot.Plotly.Interactive.dll`

Configuring PowerShell Kernel for XPlot.Plotly integration.

Installed support for XPlot.Plotly.


## Installation de OR-Tools

Pour utiliser OR-Tools avec C#, nous devons d'abord installer la bibliothèque. Vous pouvez suivre les instructions sur le site officiel de [Google OR-Tools](https://developers.google.com/optimization).

### Installation de OR-Tools

On appelle le package Nuget correspondant. 

In [2]:
#r "nuget: Google.OrTools"

Installed Packages Google.OrTools, 9.10.4067


### Liste des nutriments


| Nutriments                       | Apport quotidien recommandé         |
|----------------------------------|-------------------------------------|
| Calories                         | 3 000 calories                      |
| Protéines                        | 70 g                                |
| Calcium                          | 0,8 g                               |
| Fer                              | 12 milligrammes                     |
| Vitamine A                       | 5 000 UI                            |
| Thiamine (vitamine B1)           | 1,8 milligramme                     |
| Riboflavine (vitamine B2)        | 2,7 milligrammes                    |
| Niacine                          | 18 milligrammes                     |
| Acide ascorbique (vitamine C)    | 75 milligrammes                     |

### Liste

| Marchandises                 | Unité           | Prix 1939 (cents) | Calories (kcal) | Protéines (g) | Calcium (g) | Fer (mg) | Vitamine A (KIU) | Thiamine (mg) | Riboflavine (mg) | Niacine (mg) | Acide ascorbique (mg) |
|------------------------------|-----------------|-------------------|------------------|---------------|-------------|----------|------------------|---------------|------------------|--------------|----------------------|
| Farine de blé (enrichi)      | 4,5 kg          | 36                | 44,7             | 1411          | 2           | 365      | 0                | 55.4          | 33.3             | 441          | 0                    |
| Moutarde                     | 1 kg            | 14.1              | 11.6             | 418           | 0.7         | 54       | 0                | 3.2           | 1.9              | 68           | 0                    |
| Céréales de blé (enrichies)  | 800 g           | 24.2              | 11.8             | 377           | 14.4        | 175      | 0                | 14.4          | 8.8              | 114          | 0                    |
| Flocons de maïs              | 225 g           | 7.1               | 11.4             | 252           | 0.1         | 56       | 0                | 13.5          | 2.3              | 68           | 0                    |
| Farine de maïs               | 1 kg            | 4.6               | 36.0             | 897           | 1.7         | 99       | 30.9             | 17.4          | 7.9              | 106          | 0                    |
| Hominy Grits                 | 24 oz           | 8.5               | 28.6             | 680           | 0,8         | 80       | 0                | 10.6          | 1.6              | 110          | 0                    |
| Riz                          | 1 kg            | 7.5               | 21.2             | 460           | 0.6         | 41       | 0                | 2             | 4.8              | 60           | 0                    |
| Avoine roulée                | 1 kg            | 7.1               | 25,3             | 907           | 5.1         | 341      | 0                | 37.1          | 8.9              | 64           | 0                    |
| Pain blanc (enrichi)         | 1 kg            | 7.9               | 15.0             | 488           | 2.5         | 115      | 0                | 13.8          | 8.5              | 126          | 0                    |
| Pain au blé complet          | 1 kg            | 9.1               | 12.2             | 484           | 2.7         | 125      | 0                | 13.9          | 6.4              | 160          | 0                    |
| Pain de seigle               | 1 kg            | 9.1               | 12.4             | 439           | 1.1         | 82       | 0                | 9.9           | 3                | 66           | 0                    |
| Cake                         | 1 kg            | 24.8              | 8.0              | 130           | 0.4         | 31       | 18,9             | 2.8           | 3                | 17           | 0                    |
| Crackers à soda              | 1 kg            | 15.1              | 12.5             | 288           | 0.5         | 50       | 0                | 0             | 0                | 0            | 0                    |
| Lait                         | 1 qt            | 11                | 6.1              | 310           | 10.5        | 18       | 16,8             | 4             | 16               | 7            | 177                  |
| Lait évaporé (boîte)         | 14,5 oz         | 6.7               | 8.4              | 422           | 15.1        | 9        | 26               | 3             | 23.5             | 11           | 60                   |
| Beurre                       | 1 kg            | 30.8              | 10.8             | 9             | 0,2         | 3        | 44.2             | 0             | 0,2              | 2            | 0                    |
| Oléomargarine                | 1 kg            | 16.1              | 20,6             | 17            | 0.6         | 6        | 55.8             | 0,2           | 0                | 0            | 0                    |
| Œufs                         | 1 douz.         | 32.6              | 2.9              | 238           | 1           | 52       | 18,6             | 2.8           | 6.5              | 1            | 0                    |
| Fromage (Cheddar)            | 1 kg            | 24.2              | 7.4              | 448           | 16.4        | 19       | 28,1             | 0,8           | 10.3             | 4            | 0                    |
| Crème                        | 1/2 pt          | 14.1              | 3.5              | 49            | 1.7         | 3        | 16,9             | 0,6           | 2.5              | 0            | 17                   |
| Beurre de cacahuète          | 1 kg            | 17.9              | 15.7             | 661           | 1           | 48       | 0                | 9.6           | 8.1              | 471          | 0                    |
| Mayonnaise                   | 1/2 pt          | 16.7              | 8.6              | 18            | 0,2         | 8        | 2,7              | 0,4           | 0.5              | 0            | 0                    |
| Crisco                       | 1 kg            | 20.3              | 20.1             | 0             | 0           | 0        | 0                | 0             | 0                | 0            | 0                    |
| Lard                          | 1 kg            | 9.8               | 41.7             | 0             | 0           | 0,2      | 0                | 0             | 0.5              | 5            | 0                    |
| Faux-filet                   | 1 kg            | 39.6              | 2.9              | 166           | 0,1         | 34       | 0,2              | 2.1           | 2.9              | 69           | 0                    |
| Viande ronde                 | 1 kg            | 36.4              | 2.2              | 214           | 0,1         | 32       | 0,4              | 2.5           | 2.4              | 87           | 0                    |
| Rôti de côtelette            | 1 kg            | 29.2              | 3.4              | 213           | 0,1         | 33       | 0                | 0             | 2                | 0            | 0                    |
| Chuck roast                  | 1 kg            | 22.6              | 3.6              | 309           | 0,2         | 46       | 0,4              | 1             | 4                | 120          | 0                    |
| Assiette                     | 1 kg            | 14.6              | 8.5              | 404           | 0,2         | 62       | 0                | 0.9           | 0                | 0            | 0                    |
| Foie (bœuf)                  | 1 kg            | 26.8              | 2.2              | 333           | 0,2         | 139      | 169,2            | 6.4           | 50,8             | 316          | 525                  |
| Cuisse d'agneau              | 1 kg            | 27.6              | 3.1              | 245           | 0,1         | 20       | 0                | 2.8           | 3.9              | 86           | 0                    |
| Côtes d'agneau                | 1 kg           | 36.6              | 3.3              | 140           | 0,1         | 15       | 0                | 1.7           | 2.7              | 54           | 0                    |
| Côtes de porc                | 1 kg            | 30.7              | 3.5              | 196           | 0,2         | 30       | 0                | 17.4          | 2.7              | 60           | 0                    |
| Filet de porc rôti           | 1 kg            | 24.2              | 4.4              | 249           | 0.3         | 37       | 0                | 18.2          | 3.6              | 79           | 0                    |
| Bacon                        | 1 kg            | 25.6              | 10.4             | 152           | 0,2         | 23       | 0                | 1.8           | 1.8              | 71           | 0                    |
| Jambon fumé                  | 1 kg            | 27.4              | 6.7              | 212           | 0,2         | 31       | 0                | 9.9           | 3.3              | 50           | 0                    |
| Porc salé                    | 1 kg            | 16                | 18,8             | 164           | 0,1         | 26       | 0                | 1.4           | 1.8              | 0            | 0                    |
| Poulet rôti                  | 1 kg            | 30.3              | 1.8              | 184           | 0,1         | 30       | 0,1              | 0.9           | 1.8              | 68           | 46                   |
| Escalopes de veau            | 1 kg            | 42.3              | 1.7              | 156           | 0,1         | 24       | 0                | 1.4           | 2.4              | 57           | 0                    |
| Saumon, rose (boîte)         | 500 g           | 13                | 5.8              | 705           | 6.8         | 45       | 3.5              | 1             | 4.9              | 209          | 0                    |
| Pommes                       | 1 kg            | 4.4               | 5.8              | 27            | 0.5         | 36       | 7.3              | 3.6           | 2.7              | 5            | 544                  |
| Bananes                      | 1 kg            | 6.1               | 4.9              | 60            | 0.4         | 30       | 17.4             | 2.5           | 3.5              | 28           | 498                  |
| Citrons                      | 1 douz.         | 26                | 1                | 21            | 0.5         | 14       | 0                | 0.5           | 0                | 4            | 952                  |
| Oranges                      | 1 douz.         | 30.9              | 2.2              | 40            | 1.1         | 18       | 11.1             | 3.6           | 1.3              | 10           | 1998                 |
| Haricots verts               | 1 kg            | 7.1               | 2.4              | 138           | 3.7         | 80       | 69               | 4.3           | 5.8              | 37           | 862                  |
| Chou                         | 1 kg            | 3.7               | 2.6              | 125           | 4.0         | 36       | 7.2              | 9             | 4.5              | 26           | 5369                 |
| Carottes                     | 1 groupe        | 4.7               | 2.7              | 73            | 2.8         | 43       | 188.5            | 6.1           | 4.3              | 89           | 608                  |
| Le céleri                    | 1 tige          | 7.3               | 0.9              | 51            | 3.0         | 23       | 0.9              | 1.4           | 1.4              | 9            | 313                  |
| Laitue                       | 1 tête          | 8.2               | 0.4              | 27            | 1.1         | 22       | 112.4            | 1.8           | 3.4              | 11           | 449                  |
| Oignons                      | 1 kg            | 3.6               | 5.8              | 166           | 3.8         | 59       | 16.6             | 4.7           | 5.9              | 21           | 1184                 |
| Des pommes de terre          | 6 kg            | 34                | 14.3             | 336           | 1.8         | 118      | 6.7              | 29.4          | 7.1              | 198          | 2522                 |
| Épinards                     | 1 kg            | 8.1               | 1.1              | 106           | 0           | 138      | 918.4            | 5.7           | 13.8             | 33           | 2755                 |
| Patates douces               | 1 kg            | 5.1               | 9.6              | 138           | 2.7         | 54       | 290.7            | 8.4           | 5.4              | 83           | 1912                 |
| Pêches (boîte de conserve)  | N° 2 1/2        | 16.8              | 3.7              | 20            | 0.4         | 10       | 21.5             | 0.5           | 1                | 31           | 196                  |
| Poires (boîte)               | N° 2 1/2        | 20.4              | 3.0              | 8             | 0.3         | 8        | 0,8              | 0,8           | 0,8              | 5            | 81                   |
| Ananas (boîte)               | N° 2 1/2        | 21.3              | 2.4              | 16            | 0.4         | 8        | 2                | 2.8           | 0,8              | 7            | 399                  |
| Asperges (boîte)             | N° 2            | 27,7              | 0.4              | 33            | 0.3         | 12       | 16.3             | 1.4           | 2.1              | 17           | 272                  |
| Haricots verts (boîte)       | N° 2            | 10                | 1                | 54            | 2           | 65       | 53.9             | 1.6           | 4.3              | 32           | 431                  |
| Porc et haricots (boîte)     | 500 g           | 7.1               | 7.5              | 364           | 4           | 134      | 3.5              | 8.3           | 7.7              | 56           | 0                    |
| Maïs (boîte)                | N° 2            | 10.4              | 5.2              | 136           | 0,2         | 16       | 12                | 1.6           | 2.7              | 42           | 218                  |
| Pois (boîte)                | N° 2            | 13.8              | 2.3              | 136           | 0.6         | 45       | 34.9             | 4.9           | 2.5              | 37           | 370                  |
| Tomates (boîte)             | N° 2            | 8.6               | 1.3              | 63            | 0.7         | 38       | 53.2             | 3.4           | 2.5              | 36           | 1253                 |
| Tomates (boîte)                         | N° 2   | 8.6               | 1.3              | 63            | 0.7         | 38       | 53.2            | 3.4           | 2.5              | 36           | 1253                 |
| Soupe à la tomate (boîte de conserve)   | 300 g  | 7.6               | 1.6              | 71            | 0.6         | 43       | 57.9            | 3.5           | 2.4              | 67           | 862                  |
| Pêches, séchées                        | 1 kg   | 15.7              | 8.5              | 87            | 1.7         | 173      | 86.8            | 1.2           | 4.3              | 55           | 57                   |
| Pruneaux séchés                        | 1 kg   | 9                 | 12.8             | 99            | 2.5         | 154      | 85.7            | 3.9           | 4.3              | 65           | 257                  |
| Raisins secs et séchés                 | 500 g  | 9.4               | 13.5             | 104           | 2.5         | 136      | 4.5             | 6.3           | 1.4              | 24           | 136                  |
| Pois séchés                            | 1 kg   | 7.9               | 20.0             | 1 367         | 4.2         | 345      | 2.9             | 28.7          | 18.4             | 162          | 0                    |
| Haricots Lima secs                     | 1 kg   | 8.9               | 17.4             | 1055          | 3.7         | 459      | 5.1             | 26.9          | 38.2             | 93           | 0                    |
| Haricots de marine secs                | 1 kg   | 5.9               | 26.9             | 1691          | 11.4        | 792      | 0               | 38.4          | 24.6             | 217          | 0                    |
| Café                                   | 1 kg   | 22.4              | 0                | 0             | 0           | 0        | 0               | 4             | 5.1              | 50           | 0                    |
| Thé                                    | 1/4 lb | 17.4              | 0                | 0             | 0           | 0        | 0               | 0             | 2.3              | 42           | 0                    |
| Cacao                                  | 225 g  | 8.6               | 8.7              | 237           | 3           | 72       | 0               | 2             | 11.9             | 40           | 0                    |
| Chocolat                               | 225 g  | 16.2              | 8.0              | 77            | 1.3         | 39       | 0               | 0.9           | 3.4              | 14           | 0                    |
| Sucre                                  | 4,5 kg | 51.7              | 34.9             | 0             | 0           | 0        | 0               | 0             | 0                | 0            | 0                    |
| Sirop de maïs                          | 24 oz  | 13.7              | 14.7             | 0             | 0.5         | 74       | 0               | 0             | 0                | 5            | 0                    |
| Mélasse                                | 500 g  | 13.6              | 9.0              | 0             | 10.3        | 244      | 0               | 1.9           | 7.5              | 146          | 0                    |
| Conserves de fraises                   | 1 kg   | 20.5              | 6.4              | 11            | 0.4         | 7        | 0.2             | 0.2           | 0.4              | 3            | 0                    |

### Importer le wrapper de solution linéaire

In [3]:
using System;
using System.Collections.Generic;
using Google.OrTools.LinearSolver;

### Données concernant le problème

In [4]:
// Nutrient minimums.
(String Name, double Value)[] nutrients =
  new[] { ("Calories (kcal)", 3.0), ("Protéines (g)", 70.0),    ("Calcium (g)", 0.8),
            ("Fer (mg)", 12.0),         ("Vitamine A (kIU)", 5.0), ("Vitamine B1 (mg)", 1.8),
            ("Vitamine B2 (mg)", 2.7),  ("Niacine (mg)", 18.0),       ("Vitamine C (mg)", 75.0) };

(String Name, String Unit, double Price, double[] Nutrients)[] data = new[] {
    ("Wheat Flour (Enriched)", "10 lb.", 36, new double[] { 44.7, 1411, 2, 365, 0, 55.4, 33.3, 441, 0 }),
    ("Macaroni", "1 lb.", 14.1, new double[] { 11.6, 418, 0.7, 54, 0, 3.2, 1.9, 68, 0 }),
    ("Wheat Cereal (Enriched)", "28 oz.", 24.2, new double[] { 11.8, 377, 14.4, 175, 0, 14.4, 8.8, 114, 0 }),
    ("Corn Flakes", "8 oz.", 7.1, new double[] { 11.4, 252, 0.1, 56, 0, 13.5, 2.3, 68, 0 }),
    ("Corn Meal", "1 lb.", 4.6, new double[] { 36.0, 897, 1.7, 99, 30.9, 17.4, 7.9, 106, 0 }),
    ("Hominy Grits", "24 oz.", 8.5, new double[] { 28.6, 680, 0.8, 80, 0, 10.6, 1.6, 110, 0 }),
    ("Rice", "1 lb.", 7.5, new double[] { 21.2, 460, 0.6, 41, 0, 2, 4.8, 60, 0 }),
    ("Rolled Oats", "1 lb.", 7.1, new double[] { 25.3, 907, 5.1, 341, 0, 37.1, 8.9, 64, 0 }),
    ("White Bread (Enriched)", "1 lb.", 7.9, new double[] { 15.0, 488, 2.5, 115, 0, 13.8, 8.5, 126, 0 }),
    ("Whole Wheat Bread", "1 lb.", 9.1, new double[] { 12.2, 484, 2.7, 125, 0, 13.9, 6.4, 160, 0 }),
    ("Rye Bread", "1 lb.", 9.1, new double[] { 12.4, 439, 1.1, 82, 0, 9.9, 3, 66, 0 }),
    ("Pound Cake", "1 lb.", 24.8, new double[] { 8.0, 130, 0.4, 31, 18.9, 2.8, 3, 17, 0 }),
    ("Soda Crackers", "1 lb.", 15.1, new double[] { 12.5, 288, 0.5, 50, 0, 0, 0, 0, 0 }),
    ("Milk", "1 qt.", 11, new double[] { 6.1, 310, 10.5, 18, 16.8, 4, 16, 7, 177 }),
    ("Evaporated Milk (can)", "14.5 oz.", 6.7, new double[] { 8.4, 422, 15.1, 9, 26, 3, 23.5, 11, 60 }),
    ("Butter", "1 lb.", 30.8, new double[] { 10.8, 9, 0.2, 3, 44.2, 0, 0.2, 2, 0 }),
    ("Oleomargarine", "1 lb.", 16.1, new double[] { 20.6, 17, 0.6, 6, 55.8, 0.2, 0, 0, 0 }),
    ("Eggs", "1 doz.", 32.6, new double[] { 2.9, 238, 1.0, 52, 18.6, 2.8, 6.5, 1, 0 }),
    ("Cheese (Cheddar)", "1 lb.", 24.2, new double[] { 7.4, 448, 16.4, 19, 28.1, 0.8, 10.3, 4, 0 }),
    ("Cream", "1/2 pt.", 14.1, new double[] { 3.5, 49, 1.7, 3, 16.9, 0.6, 2.5, 0, 17 }),
    ("Peanut Butter", "1 lb.", 17.9, new double[] { 15.7, 661, 1.0, 48, 0, 9.6, 8.1, 471, 0 }),
    ("Mayonnaise", "1/2 pt.", 16.7, new double[] { 8.6, 18, 0.2, 8, 2.7, 0.4, 0.5, 0, 0 }),
    ("Crisco", "1 lb.", 20.3, new double[] { 20.1, 0, 0, 0, 0, 0, 0, 0, 0 }),
    ("Lard", "1 lb.", 9.8, new double[] { 41.7, 0, 0, 0, 0.2, 0, 0.5, 5, 0 }),
    ("Sirloin Steak", "1 lb.", 39.6, new double[] { 2.9, 166, 0.1, 34, 0.2, 2.1, 2.9, 69, 0 }),
    ("Round Steak", "1 lb.", 36.4, new double[] { 2.2, 214, 0.1, 32, 0.4, 2.5, 2.4, 87, 0 }),
    ("Rib Roast", "1 lb.", 29.2, new double[] { 3.4, 213, 0.1, 33, 0, 0, 2, 0, 0 }),
    ("Chuck Roast", "1 lb.", 22.6, new double[] { 3.6, 309, 0.2, 46, 0.4, 1, 4, 120, 0 }),
    ("Plate", "1 lb.", 14.6, new double[] { 8.5, 404, 0.2, 62, 0, 0.9, 0, 0, 0 }),
    ("Liver (Beef)", "1 lb.", 26.8, new double[] { 2.2, 333, 0.2, 139, 169.2, 6.4, 50.8, 316, 525 }),
    ("Leg of Lamb", "1 lb.", 27.6, new double[] { 3.1, 245, 0.1, 20, 0, 2.8, 3.9, 86, 0 }),
    ("Lamb Chops (Rib)", "1 lb.", 36.6, new double[] { 3.3, 140, 0.1, 15, 0, 1.7, 2.7, 54, 0 }),
    ("Pork Chops", "1 lb.", 30.7, new double[] { 3.5, 196, 0.2, 30, 0, 17.4, 2.7, 60, 0 }),
    ("Pork Loin Roast", "1 lb.", 24.2, new double[] { 4.4, 249, 0.3, 37, 0, 18.2, 3.6, 79, 0 }),
    ("Bacon", "1 lb.", 25.6, new double[] { 10.4, 152, 0.2, 23, 0, 1.8, 1.8, 71, 0 }),
    ("Ham, smoked", "1 lb.", 27.4, new double[] { 6.7, 212, 0.2, 31, 0, 9.9, 3.3, 50, 0 }),
    ("Salt Pork", "1 lb.", 16, new double[] { 18.8, 164, 0.1, 26, 0, 1.4, 1.8, 0, 0 }),
    ("Roasting Chicken", "1 lb.", 30.3, new double[] { 1.8, 184, 0.1, 30, 0.1, 0.9, 1.8, 68, 46 }),
    ("Veal Cutlets", "1 lb.", 42.3, new double[] { 1.7, 156, 0.1, 24, 0, 1.4, 2.4, 57, 0 }),
    ("Salmon, Pink (can)", "16 oz.", 13, new double[] { 5.8, 705, 6.8, 45, 3.5, 1, 4.9, 209, 0 }),
    ("Apples", "1 lb.", 4.4, new double[] { 5.8, 27, 0.5, 36, 7.3, 3.6, 2.7, 5, 544 }),
    ("Bananas", "1 lb.", 6.1, new double[] { 4.9, 60, 0.4, 30, 17.4, 2.5, 3.5, 28, 498 }),
    ("Lemons", "1 doz.", 26, new double[] { 1.0, 21, 0.5, 14, 0, 0.5, 0, 4, 952 }),
    ("Oranges", "1 doz.", 30.9, new double[] { 2.2, 40, 1.1, 18, 11.1, 3.6, 1.3, 10, 1998 }),
    ("Green Beans", "1 lb.", 7.1, new double[] { 2.4, 138, 3.7, 80, 69, 4.3, 5.8, 37, 862 }),
    ("Cabbage", "1 lb.", 3.7, new double[] { 2.6, 125, 4.0, 36, 7.2, 9, 4.5, 26, 5369 }),
    ("Carrots", "1 bunch", 4.7, new double[] { 2.7, 73, 2.8, 43, 188.5, 6.1, 4.3, 89, 608 }),
    ("Celery", "1 stalk", 7.3, new double[] { 0.9, 51, 3.0, 23, 0.9, 1.4, 1.4, 9, 313 }),
    ("Lettuce", "1 head", 8.2, new double[] { 0.4, 27, 1.1, 22, 112.4, 1.8, 3.4, 11, 449 }),
    ("Onions", "1 lb.", 3.6, new double[] { 5.8, 166, 3.8, 59, 16.6, 4.7, 5.9, 21, 1184 }),
    ("Potatoes", "15 lb.", 34, new double[] { 14.3, 336, 1.8, 118, 6.7, 29.4, 7.1, 198, 2522 }),
    ("Spinach", "1 lb.", 8.1, new double[] { 1.1, 106, 0, 138, 918.4, 5.7, 13.8, 33, 2755 }),
    ("Sweet Potatoes", "1 lb.", 5.1, new double[] { 9.6, 138, 2.7, 54, 290.7, 8.4, 5.4, 83, 1912 }),
    ("Peaches (can)", "No. 2 1/2", 16.8, new double[] { 3.7, 20, 0.4, 10, 21.5, 0.5, 1, 31, 196 }),
    ("Pears (can)", "No. 2 1/2", 20.4, new double[] { 3.0, 8, 0.3, 8, 0.8, 0.8, 0.8, 5, 81 }),
    ("Pineapple (can)", "No. 2 1/2", 21.3, new double[] { 2.4, 16, 0.4, 8, 2, 2.8, 0.8, 7, 399 }),
    ("Asparagus (can)", "No. 2", 27.7, new double[] { 0.4, 33, 0.3, 12, 16.3, 1.4, 2.1, 17, 272 }),
    ("Green Beans (can)", "No. 2", 10, new double[] { 1.0, 54, 2, 65, 53.9, 1.6, 4.3, 32, 431 }),
    ("Pork and Beans (can)", "16 oz.", 7.1, new double[] { 7.5, 364, 4, 134, 3.5, 8.3, 7.7, 56, 0 }),
    ("Corn (can)", "No. 2", 10.4, new double[] { 5.2, 136, 0.2, 16, 12, 1.6, 2.7, 42, 218 }),
    ("Peas (can)", "No. 2", 13.8, new double[] { 2.3, 136, 0.6, 45, 34.9, 4.9, 2.5, 37, 370 }),
    ("Tomatoes (can)", "No. 2", 8.6, new double[] { 1.3, 63, 0.7, 38, 53.2, 3.4, 2.5, 36, 1253 }),
    ("Tomato Soup (can)", "10 1/2 oz.", 7.6, new double[] { 1.6, 71, 0.6, 43, 57.9, 3.5, 2.4, 67, 862 }),
    ("Peaches, Dried", "1 lb.", 15.7, new double[] { 8.5, 87, 1.7, 173, 86.8, 1.2, 4.3, 55, 57 }),
    ("Prunes, Dried", "1 lb.", 9, new double[] { 12.8, 99, 2.5, 154, 85.7, 3.9, 4.3, 65, 257 }),
    ("Raisins, Dried", "15 oz.", 9.4, new double[] { 13.5, 104, 2.5, 136, 4.5, 6.3, 1.4, 24, 136 }),
    ("Peas, Dried", "1 lb.", 7.9, new double[] { 20.0, 1367, 4.2, 345, 2.9, 28.7, 18.4, 162, 0 }),
    ("Lima Beans, Dried", "1 lb.", 8.9, new double[] { 17.4, 1055, 3.7, 459, 5.1, 26.9, 38.2, 93, 0 }),
    ("Navy Beans, Dried", "1 lb.", 5.9, new double[] { 26.9, 1691, 11.4, 792, 0, 38.4, 24.6, 217, 0 }),
    ("Coffee", "1 lb.", 22.4, new double[] { 0, 0, 0, 0, 0, 4, 5.1, 50, 0 }),
    ("Tea", "1/4 lb.", 17.4, new double[] { 0, 0, 0, 0, 0, 0, 2.3, 42, 0 }),
    ("Cocoa", "8 oz.", 8.6, new double[] { 8.7, 237, 3, 72, 0, 2, 11.9, 40, 0 }),
    ("Chocolate", "8 oz.", 16.2, new double[] { 8.0, 77, 1.3, 39, 0, 0.9, 3.4, 14, 0 }),
    ("Sugar", "10 lb.", 51.7, new double[] { 34.9, 0, 0, 0, 0, 0, 0, 0, 0 }),
    ("Corn Syrup", "24 oz.", 13.7, new double[] { 14.7, 0, 0.5, 74, 0, 0, 0, 5, 0 }),
    ("Molasses", "18 oz.", 13.6, new double[] { 9.0, 0, 10.3, 244, 0, 1.9, 7.5, 146, 0 }),
    ("Strawberry Preserves", "1 lb.", 20.5, new double[] { 6.4, 11, 0.4, 7, 0.2, 0.2, 0.4, 3, 0 })
};

### Déclarer le résolveur LP

In [5]:
// Créer le solveur linéaire avec le backend GLOP.
Solver solver = Solver.CreateSolver("GLOP");
if (solver is null)
{
    return;
}

<null>

### Créer les variables

In [6]:
// Créer les variables 

List<Variable> foods = new List<Variable>();
for (int i = 0; i < data.Length; ++i)
{
    foods.Add(solver.MakeNumVar(0.0, double.PositiveInfinity, data[i].Name));
}
Console.WriteLine($"Nombre de variables = {solver.NumVariables()}");

Nombre de variables = 77


### Définir les contraintes & Créer l'objectif

In [7]:
// définir les contraintes
List<Constraint> constraints = new List<Constraint>();
for (int i = 0; i < nutrients.Length; ++i)
{
    Constraint constraint =
        solver.MakeConstraint(nutrients[i].Value, double.PositiveInfinity, nutrients[i].Name);
    for (int j = 0; j < data.Length; ++j)
    {
        constraint.SetCoefficient(foods[j], data[j].Nutrients[i]);
    }
    constraints.Add(constraint);
}
Console.WriteLine($"Nombre de contraintes = {solver.NumConstraints()}");


// déclaration des objectifs 
Objective objective = solver.Objective();
for (int i = 0; i < data.Length; ++i)
{
    objective.SetCoefficient(foods[i], 1);
}
objective.SetMinimization();

Nombre de contraintes = 9


### Appeler le résolveur & Afficher la solution

In [8]:
Solver.ResultStatus resultStatus = solver.Solve();

// Vérifie que le problème possède une solution optimale.
if (resultStatus != Solver.ResultStatus.OPTIMAL)
{
    Console.WriteLine("Le problème n'a pas de solution optimale !");
    if (resultStatus == Solver.ResultStatus.FEASIBLE)
    {
        Console.WriteLine("Une solution potentiellement sous-optimale a été trouvée.");
    }
    else
    {
        Console.WriteLine("Le solveur n'a pas pu résoudre le problème.");
        return;
    }
}

// Affiche les montants (en dollars) à acheter de chaque aliment.
double[] nutrientsResult = new double[nutrients.Length];
Console.WriteLine("\nAliments annuels:");
for (int i = 0; i < foods.Count; ++i)
{
    if (foods[i].SolutionValue() > 0.0)
    {
        Console.WriteLine($"{data[i].Name}: ${365 * foods[i].SolutionValue():N2}");
        for (int j = 0; j < nutrients.Length; ++j)
        {
            nutrientsResult[j] += data[i].Nutrients[j] * foods[i].SolutionValue();
        }
    }
}
Console.WriteLine($"\nPrix annuel optimal : ${365 * objective.Value():N2}");

Console.WriteLine("\nNutriments par jour :");
for (int i = 0; i < nutrients.Length; ++i)
{
    Console.WriteLine($"{nutrients[i].Name}: {nutrientsResult[i]:N2} (min {nutrients[i].Value})");
}


Aliments annuels:
Wheat Flour (Enriched): $10,77
Liver (Beef): $0,69
Cabbage: $4,09
Spinach: $1,83
Navy Beans, Dried: $22,28

Prix annuel optimal : $39,66

Nutriments par jour :
Calories (kcal): 3,00 (min 3)
Protéines (g): 147,41 (min 70)
Calcium (g): 0,80 (min 0,8)
Fer (mg): 60,47 (min 12)
Vitamine A (kIU): 5,00 (min 5)
Vitamine B1 (mg): 4,12 (min 1,8)
Vitamine B2 (mg): 2,70 (min 2,7)
Niacine (mg): 27,32 (min 18)
Vitamine C (mg): 75,00 (min 75)
